In [18]:
from prepared_data.completed_missing_data import completed_missing_data, convert_historical_date_to_date_difference, remove_described_col_and_set_index_id, map_target
import pandas as pd
import numpy as np


In [19]:
train_org = pd.read_csv('../data/train.csv')
train = completed_missing_data(train_org)
train = convert_historical_date_to_date_difference(train)
train = map_target(train)


/home/andrzej/PycharmProjects/Physionet_Challenge_2021/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


correction col type..
percent of object with nan value:  42.20
remove coach columns..
percent of object with nan value:  18.64
remove object with more that  col 80 empty...
percent of object with nan value:  18.64
remove 9 and 10 day historii
percent of object with nan value:  15.11
remove all nan data
percent of object with nan value:  15.11


In [20]:
test_org = pd.read_csv('../data/test.csv')
test = completed_missing_data(test_org)
test = convert_historical_date_to_date_difference(test)
test = remove_described_col_and_set_index_id(test)
test

correction col type..
percent of object with nan value:  39.35
remove coach columns..
percent of object with nan value:  13.93
remove object with more that  col 80 empty...
percent of object with nan value:  13.93
remove 9 and 10 day historii
percent of object with nan value:  11.15
remove all nan data
percent of object with nan value:  11.15


,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,home_team_history_is_play_home_1,...,away_team_history_rating_7,away_team_history_rating_8,away_team_history_opponent_rating_1,away_team_history_opponent_rating_2,away_team_history_opponent_rating_3,away_team_history_opponent_rating_4,away_team_history_opponent_rating_5,away_team_history_opponent_rating_6,away_team_history_opponent_rating_7,away_team_history_opponent_rating_8
id,,,,,,,,,,,,,,,,,,,,,
17761448,0,71,97,70,96,143,98,120,288,0.0,...,4.972890,3.150812,4.721590,6.953965,6.535228,5.782611,7.538950,8.428800,4.922229,6.796484
17695487,0,166,169,168,143,360,94,193,72,0.0,...,6.765418,6.203948,4.459100,4.543053,4.861583,5.082300,9.011000,6.621075,4.141575,5.056690
17715493,0,48,98,46,48,918,72,97,95,0.0,...,8.730013,7.177425,3.790810,3.562960,4.655450,4.232500,3.999711,4.095675,3.746925,4.952475
17715492,0,48,98,48,48,917,68,100,95,0.0,...,6.550694,3.018285,6.000775,7.648230,5.580242,6.971550,5.624680,5.733725,4.453211,8.787767
17715490,0,48,98,46,50,893,96,96,119,0.0,...,3.746925,4.472350,5.304272,7.069420,6.717443,6.130100,5.281494,5.731775,8.730013,6.617900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18450246,0,145,163,169,190,122,216,142,219,1.0,...,7.010503,6.770769,5.418217,4.389327,4.568866,6.076045,3.720895,4.656597,4.494093,3.665037
18164889,0,141,98,286,120,96,144,95,97,0.0,...,4.458606,5.017476,4.839404,5.167733,5.403118,5.803769,5.873700,5.143589,6.695206,6.109570
18449018,0,913,765,1875,93,450,1452,77,1097,1.0,...,12.404786,5.000000,4.828760,4.448000,8.047350,7.526000,7.597350,6.304886,2.612821,5.000000


# Dodanie cech
- Sredni ranking drużyn w lidzie
- Sredni ranking ligi wygranych w lidze
- Intensywnosc druzyny, na podstawie dat rozegranych meczów
- Historyczny mecz target, na postawie bramek




In [21]:
def team_regeneration(data_f, team_h_or_a, index, history_matches_amount):
    i_str =str(index)
    if index == 1:
        data_f[team_h_or_a + '_team_mean_regeneration_time'] = data_f[team_h_or_a + '_team_history_match_date_' + i_str] / history_matches_amount
    else:
        data_f[team_h_or_a + '_team_mean_regeneration_time'] += data_f[team_h_or_a + '_team_history_match_date_' + i_str] / history_matches_amount
    return data_f


def history_target(data_f, team_h_or_a, index):
    i_str = str(index)
    team_history_goal = team_h_or_a + '_team_history_goal_' + i_str

    team_history_opponent_goal = team_h_or_a + '_team_history_opponent_goal_' + i_str


    data_f[team_h_or_a + '_team_history_gol_difference_' + i_str] = data_f[team_history_goal] - data_f[team_history_opponent_goal]

    data_f[team_h_or_a + '_team_history_target_' + i_str] = np.sign(
        data_f[team_history_goal] - data_f[team_history_opponent_goal])
    return  data_f




def league_team_mean_ratting(data_f, team_h_or_a, index):
    i_str = str(index)
    last_match_columns = [x for x in data_f.columns if i_str in x]
    rating_col_name = team_h_or_a + '_team_history_league_id_' + i_str

    league_mean_ratting = data_f[last_match_columns].groupby(by=rating_col_name).mean()
    league_mean_ratting['mean_ratting_' + team_h_or_a + '_' + i_str] = (league_mean_ratting[team_h_or_a + '_team_history_rating_' + i_str]
                                                                      + league_mean_ratting[team_h_or_a + '_team_history_opponent_rating_' + i_str]) / 2

    return league_mean_ratting['mean_ratting_' + team_h_or_a + '_' + i_str]

def teams_mean_ratting(data_f, team_h_or_a, index):
    i_str = str(index)
    team_col_name = team_h_or_a+'_team_name'

    team_mean_ratting = data_f.groupby(by=team_col_name).mean()

    return team_mean_ratting[team_h_or_a + '_team_history_rating_' + i_str]

league_mean_ratting_all = pd.DataFrame()
teams_mean_ratting_all = pd.DataFrame()

number_of_history_matches = 8

for i in range(1, number_of_history_matches + 1):
    for home_or_away in ['home', 'away']:

        league_mean_ratting_all = league_mean_ratting_all.append(league_team_mean_ratting(train, home_or_away, i))
        teams_mean_ratting_all = teams_mean_ratting_all.append(teams_mean_ratting(train,home_or_away,i))



        train = history_target(train, home_or_away, i)
        train = team_regeneration(train, home_or_away, i,number_of_history_matches)

        if i == number_of_history_matches-1:
            team_name_col = home_or_away + '_team_name'
            teams_ratting = teams_mean_ratting_all.mean().reset_index().rename(columns = {'index': team_name_col, 0: home_or_away+'_team_mean_ratting'})
            train = train.merge(teams_ratting, on=team_name_col)

league_mean_ratting_all = league_mean_ratting_all.mean().reset_index().rename(
    columns={'index': 'league_id', 0: 'league_id_ratting'})

train = train.merge(league_mean_ratting_all, on='league_id').sort_values(by='league_id')

train

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,...,home_team_history_target_7,home_team_mean_ratting,away_team_history_gol_difference_7,away_team_history_target_7,away_team_mean_ratting,home_team_history_gol_difference_8,home_team_history_target_8,away_team_history_gol_difference_8,away_team_history_target_8,league_id_ratting
23020,11989265,-1,Tottenham Hotspur,RB Leipzig,2020-02-19 20:00:00,Champions League,2,1,78,258,...,1.0,8.243467,2.0,1.0,9.009022,-1.0,-1.0,0.0,0.0,6.805677
23092,16865249,1,Celtic,KR,2020-08-18 18:45:00,Champions League,2,1,219,168,...,0.0,10.467791,2.0,1.0,9.855052,1.0,1.0,1.0,1.0,6.805677
23091,16916583,1,Legia Warszawa,Linfield,2020-08-18 17:00:00,Champions League,2,1,94,627,...,0.0,8.842619,-2.0,-1.0,9.742074,0.0,0.0,7.0,1.0,6.805677
23090,16940078,-1,Sūduva,Maccabi Tel Aviv,2020-08-26 16:00:00,Champions League,2,1,167,168,...,1.0,9.933509,3.0,1.0,8.395570,3.0,1.0,2.0,1.0,6.805677
23089,17213503,1,Salzburg,Maccabi Tel Aviv,2020-09-30 19:00:00,Champions League,2,1,100,92,...,0.0,11.734431,2.0,1.0,8.395570,-3.0,-1.0,-2.0,-1.0,6.805677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42463,17882059,1,Alkmaar W,Excelsior Barendrecht W,2021-02-26 18:30:00,Eredivisie Cup,2037,1,504,1135,...,0.0,6.549518,0.0,0.0,5.621235,0.0,0.0,-3.0,-1.0,7.900250
42462,17887522,-1,SC Heerenveen W,Alkmaar W,2021-04-02 17:30:00,Eredivisie Cup,2037,1,127,167,...,-1.0,6.748837,-3.0,-1.0,6.489733,-2.0,-1.0,-1.0,-1.0,7.900250
42461,17882061,-1,SC Heerenveen W,Ajax W,2021-02-28 11:30:00,Eredivisie Cup,2037,1,545,1133,...,-1.0,6.748837,1.0,1.0,11.682420,-2.0,-1.0,3.0,1.0,7.900250
42460,17882060,1,PEC Zwolle W,ADO Den Haag W,2021-02-26 18:30:00,Eredivisie Cup,2037,1,504,1344,...,-1.0,6.834174,2.0,1.0,9.663588,0.0,0.0,2.0,1.0,7.900250


In [22]:
target_columns = [x for x in train.columns if 'history_target' in x]
train = pd.get_dummies(train, columns=target_columns)

In [23]:
train = remove_described_col_and_set_index_id(train)
# %%
train

,target,is_cup,home_team_history_match_date_1,home_team_history_match_date_2,home_team_history_match_date_3,home_team_history_match_date_4,home_team_history_match_date_5,home_team_history_match_date_6,home_team_history_match_date_7,home_team_history_match_date_8,...,home_team_history_target_7_1.0,away_team_history_target_7_-1.0,away_team_history_target_7_0.0,away_team_history_target_7_1.0,home_team_history_target_8_-1.0,home_team_history_target_8_0.0,home_team_history_target_8_1.0,away_team_history_target_8_-1.0,away_team_history_target_8_0.0,away_team_history_target_8_1.0
id,,,,,,,,,,,,,,,,,,,,,
11989265,-1,1,78,258,75,193,67,103,88,74,...,1,0,0,1,1,0,0,0,1,0
16865249,1,1,219,168,286,71,49,3145,67,76,...,0,0,0,1,0,0,1,0,0,1
16916583,1,1,94,627,93,99,93,74,168,69,...,0,1,0,0,0,1,0,0,0,1
16940078,-1,1,167,168,119,123,117,121,96,96,...,1,0,0,1,0,0,1,0,0,1
17213503,1,1,100,92,76,144,92,126,238,72,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17882059,1,1,504,1135,712,168,168,168,505,168,...,0,0,1,0,0,1,0,1,0,0
17887522,-1,1,127,167,208,72,96,127,545,1133,...,0,1,0,0,1,0,0,1,0,0
17882061,-1,1,545,1133,715,168,168,168,505,168,...,0,0,0,1,1,0,0,0,0,1


In [24]:
import re
used = set()
columns_only_first_hist = [column for column in train.columns.values if
                           re.sub('\d', '', column) not in used and (used.add(re.sub('\d', '', column)) or True)]
print(columns_only_first_hist)

['target', 'is_cup', 'home_team_history_match_date_1', 'home_team_history_is_play_home_1', 'home_team_history_is_cup_1', 'home_team_history_goal_1', 'home_team_history_opponent_goal_1', 'home_team_history_rating_1', 'home_team_history_opponent_rating_1', 'away_team_history_match_date_1', 'away_team_history_is_play_home_1', 'away_team_history_is_cup_1', 'away_team_history_goal_1', 'away_team_history_opponent_goal_1', 'away_team_history_rating_1', 'away_team_history_opponent_rating_1', 'home_team_history_gol_difference_1', 'home_team_mean_regeneration_time', 'away_team_history_gol_difference_1', 'away_team_mean_regeneration_time', 'home_team_mean_ratting', 'away_team_mean_ratting', 'league_id_ratting', 'home_team_history_target_1_-1.0', 'home_team_history_target_1_0.0', 'away_team_history_target_1_-1.0', 'away_team_history_target_1_0.0']


In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

for data, name in zip([train], ['train']):
    X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.33,
                                                        random_state=42)
    pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

    pipe.fit(X_train, y_train)
    results = pipe.predict(X_train)
    print(f'accuracy: {pipe.score(X_test, y_test):.02f}')
    print(name, results)
# drzewach, svm
# kolumny
# kaggle
#

accuracy: 0.50
train [ 1  1  1 ...  1 -1  1]


# Dobór cech

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

svc = make_pipeline(StandardScaler(), LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr'))

sfs = SequentialFeatureSelector(svc, n_features_to_select=5,cv= 5)
sfs.fit(X_train, y_train)

X_train.columns[ sfs.get_support()]
